In [1]:
import pandas as pd
from lxml import etree
import re
from thefuzz import fuzz

In [2]:
#ouverture du fichier XML grâce à etree

xml_file = '/Users/margotpascual/Documents/ENC/dico-topo/data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

#xpath pour sélectionner tous les articles + regex pour s'occuper des balises XML

xpath_expr = '//article'

pattern = re.compile(r'<.*?>')

#on crée une liste vide pour ensuite mettre toutes les valeurs nécessaires au dataframe dans un dictionnaire par la suite

data = []

#on s'occupe des vedettes: pour l'instant, notre code ne prend pas en compte les vedettes avec + d'une valeur. A voir
#si il faut changer ça. sur le xpath cependant, on dirait qu'il n'y en a pas ?
for article in xml_tree.xpath(xpath_expr):

#on enleve les localisationpa : inutiles pour nous si on ne se focalise que sur la france.
        if article.find('./localisationpa') is not None:
             continue
        vedette_i = article.xpath('./vedette/i')
        vedette_text = vedette_i[0].text if vedette_i else ''

#pareil: il y a un seul localisationde en double, mais il ne le prend pas en compte. A voir si il faut changer ça
        localisationde = article.find('./localisationde')
        localisationde_text = localisationde.text if localisationde is not None else 'none'

#idem. autre version du code qui en prend deux et sépare par ',' existe, à voir si c'est vraiment pertinent.
        localisationca = article.find('./localisationca')
        if localisationca is not None:
            localisationca_text = re.search(r'<localisationca>(.*?)<\/localisationca>', etree.tostring(localisationca, encoding=str)).group(1)
            localisationca_text = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca_text)
            
            '''
            localisationca_text = localisationca_text.replace('c<sup>on</sup> de ', '')
            localisationca_text = localisationca_text.replace('c<sup>on</sup> du ', '')
            localisationca_text = localisationca_text.replace("c<sup>on</sup> d’", '')
            localisationca_text = localisationca_text.replace('c<sup>on</sup> de la ', '')
            localisationca_text = localisationca_text.replace('c<sup>on</sup> de l’', '')
            localisationca_text = localisationca_text.replace('c<sup>on</sup> des ', '')
            '''
            
        else:
            localisationca_text = 'none'

#idem. Il n'y en a qu'un avec + d'une localisation -> l'autre étant en suisse. Utile ou pas? à voir.
        localisationco = article.find('./localisationco')
        if localisationco is not None:
            localisationco_text = re.search(r'<localisationco>(.*?)<\/localisationco>', etree.tostring(localisationco, encoding=str)).group(1)
            localisationco_text = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco_text)
            
            '''
            localisationco_text = localisationco_text.replace('c<sup>ne</sup> de ', '')
            localisationco_text = localisationco_text.replace('c<sup>ne</sup> du ', '')
            localisationco_text = localisationco_text.replace("c<sup>ne</sup> d’", '')
            localisationco_text = localisationco_text.replace('c<sup>ne</sup> de la ', '')
            localisationco_text = localisationco_text.replace('c<sup>ne</sup> de l’', '')
            localisationco_text = localisationco_text.replace('c<sup>ne</sup> des ', '')
            localisationco_text = localisationco_text.replace('c<sup>nes</sup> de ', '')
            localisationco_text = localisationco_text.replace('c<sup>nes</sup> d’', '')
            '''
            
        else: 
            localisationco_text = 'none'
    
#on insère tout dans un dictionnaire        
        data.append({
            'old-id': article.get('old-id'),
            'vedette': vedette_text,
            'localisationde': localisationde_text,
            'localisationca': localisationca_text,
            'localisationco': localisationco_text
        })

In [3]:
#on crée notre première dataframe issue de notre parsing XML

df = pd.DataFrame(data)

df.head(25)

,old-id,vedette,localisationde,localisationca,localisationco
0,PO7-00001,Abbaye,Drôme,Die,Die
1,PO7-00002,Abbaye,Isère,Roybon,Marnans
2,PO7-00003,Abbaye,Haute-Savoie,Biot,Saint-Jean-d’Aulph
3,PO7-00005,Abbaye-d’Acey,Jura,Gendrey,Ougney
4,PO7-00006,Abbaye-de-Grandvaux,Jura,Saint-Laurent,Rivière-Devant
5,PO7-00007,Abbaye-de-la-Charité,Haute-Saône,Scey-sur-Saône,Neuvelle-lès-la-Charité
6,PO7-00008,Abbaye-des-Trois-Rois,Doubs,l’Isle-sur-le-Doubs,Mancenans
7,PO7-00009,Abbenans,Doubs,Rougemont,none
8,PO7-00010,Abbevillers,Doubs,Hérimoncourt,none
9,PO7-00011,Aboncourt,Haute-Saône,Combeaufontaine,none


In [4]:
departements_df = pd.read_csv('departements-region.csv')

# On crée un dictionnaire avec les éléments qui nous intéressent de notre spreadsheet des départements
departements_dict = dict(zip(departements_df['dep_name'], departements_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['num_dep'] = df['localisationde'].apply(lambda x: departements_dict.get(x, 'none'))

print(df)

         old-id              vedette localisationde localisationca  \
0     PO7-00001               Abbaye          Drôme            Die   
1     PO7-00002               Abbaye          Isère         Roybon   
2     PO7-00003               Abbaye   Haute-Savoie           Biot   
3     PO7-00005        Abbaye-d’Acey           Jura        Gendrey   
4     PO7-00006  Abbaye-de-Grandvaux           Jura  Saint-Laurent   
...         ...                  ...            ...            ...   
3423  PO7-04280           Zaessingue      Haut-Rhin        Landser   
3424  PO7-04282           Zellenberg      Haut-Rhin    Kaysersberg   
3425  PO7-04285           Zillisheim      Haut-Rhin       Mulhouse   
3426  PO7-04286           Zimmerbach      Haut-Rhin    Wintzenheim   
3427  PO7-04287          Zimmersheim      Haut-Rhin       Habsheim   

          localisationco num_dep  
0                    Die      26  
1                Marnans      38  
2     Saint-Jean-d’Aulph      74  
3                 O

In [5]:
#on renomme pour plus de clarté.

df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune', 'num_dep' : 'dpt_code'})
df

,article_id,vedette,localisationde,canton_code,localisationco,dpt_code
0,PO7-00001,Abbaye,Drôme,Die,Die,26
1,PO7-00002,Abbaye,Isère,Roybon,Marnans,38
2,PO7-00003,Abbaye,Haute-Savoie,Biot,Saint-Jean-d’Aulph,74
3,PO7-00005,Abbaye-d’Acey,Jura,Gendrey,Ougney,39
4,PO7-00006,Abbaye-de-Grandvaux,Jura,Saint-Laurent,Rivière-Devant,39
...,...,...,...,...,...,...
3423,PO7-04280,Zaessingue,Haut-Rhin,Landser,none,68
3424,PO7-04282,Zellenberg,Haut-Rhin,Kaysersberg,none,68
3425,PO7-04285,Zillisheim,Haut-Rhin,Mulhouse,none,68
3426,PO7-04286,Zimmerbach,Haut-Rhin,Wintzenheim,none,68


In [6]:
#pour encore plus de clarté, nous réorganisons les colonnes 

df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco']]
df

,article_id,vedette,localisationde,dpt_code,canton_code,localisationco
0,PO7-00001,Abbaye,Drôme,26,Die,Die
1,PO7-00002,Abbaye,Isère,38,Roybon,Marnans
2,PO7-00003,Abbaye,Haute-Savoie,74,Biot,Saint-Jean-d’Aulph
3,PO7-00005,Abbaye-d’Acey,Jura,39,Gendrey,Ougney
4,PO7-00006,Abbaye-de-Grandvaux,Jura,39,Saint-Laurent,Rivière-Devant
...,...,...,...,...,...,...
3423,PO7-04280,Zaessingue,Haut-Rhin,68,Landser,none
3424,PO7-04282,Zellenberg,Haut-Rhin,68,Kaysersberg,none
3425,PO7-04285,Zillisheim,Haut-Rhin,68,Mulhouse,none
3426,PO7-04286,Zimmerbach,Haut-Rhin,68,Wintzenheim,none


In [7]:
#on ne garde que les lieux avec une localisationco

filtered_df = df[df['localisationco'] != 'none']
filtered_df['dpt_code'] = filtered_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)

print(filtered_df)

     article_id              vedette localisationde dpt_code    canton_code  \
0     PO7-00001               Abbaye          Drôme   DEP_26            Die   
1     PO7-00002               Abbaye          Isère   DEP_38         Roybon   
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74           Biot   
3     PO7-00005        Abbaye-d’Acey           Jura   DEP_39        Gendrey   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39  Saint-Laurent   
...         ...                  ...            ...      ...            ...   
3401  PO7-04245             Weinbach      Haut-Rhin   DEP_68    Kaysersberg   
3402  PO7-04247           Weisskirch      Haut-Rhin   DEP_68       Huningue   
3411  PO7-04259          Wintzfelden      Haut-Rhin   DEP_68       Rouffach   
3416  PO7-04268           Wolfenheim      Haut-Rhin   DEP_68         Colmar   
3421  PO7-04276                  Yon            Ain   DEP_01      Champagne   

              localisationco  
0                   

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/2923801116.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['dpt_code'] = filtered_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)


In [8]:
#On lie notre dataframe au référentiel insee avec de l'exact match.

main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')

# on utilise merge pour le liage avec dpt_code et dep_id
merged_df = pd.merge(filtered_df, main_insee_commune, left_on='dpt_code', right_on='DEP_id', how='left')

# on sélectionne les lignes où la colonne 'localisationco' correspond à la colonne 'NCCENR' de main_insee_commune
matching_rows = merged_df.loc[merged_df['localisationco'] == merged_df['NCCENR']]

# on crée une colonne 'insee_code' dans le dataframe filtered_df avec les codes correspondants à partir de main_insee_commune
filtered_df['insee_code'] = filtered_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values) > 0 else None, axis=1)
filtered_df['insee_code'] = filtered_df['insee_code'].fillna('none')


/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/1900293793.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['insee_code'] = filtered_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values) > 0 else None, axis=1)
/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/1900293793.py:13: SettingWithCopyWarning:

In [9]:
print(filtered_df)

     article_id              vedette localisationde dpt_code    canton_code  \
0     PO7-00001               Abbaye          Drôme   DEP_26            Die   
1     PO7-00002               Abbaye          Isère   DEP_38         Roybon   
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74           Biot   
3     PO7-00005        Abbaye-d’Acey           Jura   DEP_39        Gendrey   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39  Saint-Laurent   
...         ...                  ...            ...      ...            ...   
3401  PO7-04245             Weinbach      Haut-Rhin   DEP_68    Kaysersberg   
3402  PO7-04247           Weisskirch      Haut-Rhin   DEP_68       Huningue   
3411  PO7-04259          Wintzfelden      Haut-Rhin   DEP_68       Rouffach   
3416  PO7-04268           Wolfenheim      Haut-Rhin   DEP_68         Colmar   
3421  PO7-04276                  Yon            Ain   DEP_01      Champagne   

              localisationco insee_code  
0        

In [10]:
#tous nos article_id qui correspondent à un code insee ; on peut donc en conclure que ce sont effectivement des 
#communes.

commune = filtered_df[filtered_df['insee_code'] != 'none']
print(commune)

#on a donc exactement 556 matches.

     article_id                vedette localisationde dpt_code  \
0     PO7-00001                 Abbaye          Drôme   DEP_26   
1     PO7-00002                 Abbaye          Isère   DEP_38   
3     PO7-00005          Abbaye-d’Acey           Jura   DEP_39   
5     PO7-00007   Abbaye-de-la-Charité    Haute-Saône   DEP_70   
6     PO7-00008  Abbaye-des-Trois-Rois          Doubs   DEP_25   
...         ...                    ...            ...      ...   
3401  PO7-04245               Weinbach      Haut-Rhin   DEP_68   
3402  PO7-04247             Weisskirch      Haut-Rhin   DEP_68   
3411  PO7-04259            Wintzfelden      Haut-Rhin   DEP_68   
3416  PO7-04268             Wolfenheim      Haut-Rhin   DEP_68   
3421  PO7-04276                    Yon            Ain   DEP_01   

              canton_code           localisationco insee_code  
0                     Die                      Die      26113  
1                  Roybon                  Marnans      38221  
3              

In [11]:
#modification de notre dataframe: ajout d'une colonne method qui nous permet d'identifier la méthode utilisée pour le
#liage

commune['method'] = 'exact'
commune.loc[commune['dpt_code'] == 'none', 'method'] = 'nodpt'

print(commune)

     article_id                vedette localisationde dpt_code  \
0     PO7-00001                 Abbaye          Drôme   DEP_26   
1     PO7-00002                 Abbaye          Isère   DEP_38   
3     PO7-00005          Abbaye-d’Acey           Jura   DEP_39   
5     PO7-00007   Abbaye-de-la-Charité    Haute-Saône   DEP_70   
6     PO7-00008  Abbaye-des-Trois-Rois          Doubs   DEP_25   
...         ...                    ...            ...      ...   
3401  PO7-04245               Weinbach      Haut-Rhin   DEP_68   
3402  PO7-04247             Weisskirch      Haut-Rhin   DEP_68   
3411  PO7-04259            Wintzfelden      Haut-Rhin   DEP_68   
3416  PO7-04268             Wolfenheim      Haut-Rhin   DEP_68   
3421  PO7-04276                    Yon            Ain   DEP_01   

              canton_code           localisationco insee_code method  
0                     Die                      Die      26113  exact  
1                  Roybon                  Marnans      38221  ex

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/3063545279.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune['method'] = 'exact'
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [12]:
#les articles qui du coup n'ont pas de liage. on fera le fuzzy pour eux.

pascommune = filtered_df.loc[filtered_df['insee_code'] == 'none']
print(pascommune)

#nous en avons donc 124.

     article_id              vedette localisationde dpt_code      canton_code  \
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74             Biot   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39    Saint-Laurent   
13    PO7-00015               Achard          Isère   DEP_38  Villard-de-Lans   
47    PO7-00054              Alençon          Drôme   DEP_26        Dieulefit   
92    PO7-00106             Annegray    Haute-Saône   DEP_70        Faucogney   
...         ...                  ...            ...      ...              ...   
3242  PO7-04035             Vermelle          Isère   DEP_38         Bourgoin   
3243  PO7-04037            Vernaison          Drôme   DEP_26   Bourg-de-Péage   
3264  PO7-04062               Vetraz   Haute-Savoie   DEP_74        Annemasse   
3310  PO7-04128                Ville          Isère   DEP_38   la Verpillière   
3316  PO7-04134           Villeneuve          Isère   DEP_38           Domène   

              localisationc

In [13]:
# Définition d'une fonction pour le fuzzy join

def fuzzy_join(row):
    # Filtre sur les lignes avec les mêmes départements
    main_dep = main_insee_commune[main_insee_commune['DEP_id'] == row['dpt_code']]
    # Calcul de la distance Levenshtein entre la vedette du pascommune et les NCCENR de chaque ligne de main_dep
    scores = main_dep['NCCENR'].apply(lambda x: fuzz.token_sort_ratio(x, row['vedette']))
    # Récupération des lignes avec une distance Levenshtein inférieure ou égale à 1
    filtered = main_dep[scores >= 90]
    # Ajout de la colonne insee_code dans le dataframe pascommune
    if not filtered.empty:
        insee_code = filtered['insee_code'].values[0]
        if pd.isnull(insee_code):
            return 'none'
        else:
            return insee_code
    else:
        return 'none'

# Application de la fonction sur chaque ligne du dataframe pascommune
pascommune['insee_code'] = pascommune.apply(fuzzy_join, axis=1)

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/3531810569.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pascommune['insee_code'] = pascommune.apply(fuzzy_join, axis=1)


In [14]:
print(pascommune)

     article_id              vedette localisationde dpt_code      canton_code  \
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74             Biot   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39    Saint-Laurent   
13    PO7-00015               Achard          Isère   DEP_38  Villard-de-Lans   
47    PO7-00054              Alençon          Drôme   DEP_26        Dieulefit   
92    PO7-00106             Annegray    Haute-Saône   DEP_70        Faucogney   
...         ...                  ...            ...      ...              ...   
3242  PO7-04035             Vermelle          Isère   DEP_38         Bourgoin   
3243  PO7-04037            Vernaison          Drôme   DEP_26   Bourg-de-Péage   
3264  PO7-04062               Vetraz   Haute-Savoie   DEP_74        Annemasse   
3310  PO7-04128                Ville          Isère   DEP_38   la Verpillière   
3316  PO7-04134           Villeneuve          Isère   DEP_38           Domène   

              localisationc

In [15]:
#on veut savoir combien ont été joined par le fuzzy

fuzzy = pascommune[pascommune['insee_code'] != 'none']

# on ajoute une colonne method pour indiquer que le join a été réalisé par fuzzy

fuzzy['method'] = 'fuzzy'

print(fuzzy)

     article_id       vedette localisationde dpt_code          canton_code  \
124   PO7-00143      Aresches           Jura   DEP_39               Salins   
382   PO7-00476     Bonnevaux           Gard   DEP_30             Génolhac   
913   PO7-01136          Cusy   Haute-Savoie   DEP_74             Douvaine   
1181  PO7-01486        Funans          Doubs   DEP_25  l’Isle-sur-le-Doubs   
2361  PO7-02972  Saint-Andéol          Drôme   DEP_26             Marsanne   

            localisationco insee_code method  
124      Moutaine-Aresches      39586  fuzzy  
382   Bonnevaux-et-Hiverne      30044  fuzzy  
913                  Chens      74097  fuzzy  
1181   l’Isle-sur-le-Doubs      25262  fuzzy  
2361      la Bâtie-Rolland      26291  fuzzy  


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/3576432877.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzzy['method'] = 'fuzzy'


In [16]:
# on ajoute une colonne method pour indiquer que le join a été réalisé par fuzzy

fuzzy['method'] = 'fuzzy'

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/2359727262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzzy['method'] = 'fuzzy'


In [17]:
# maintenant, on s'intéresse aux localisationco qui n'ont toujours pas de liage.

fuzzynull = pascommune[pascommune['insee_code'] == 'none']

#idem, on ajoute la méthode pour savoir que ces derniers n'ont été matchés ni avec le exact ni avec le fuzzy

fuzzynull['method'] = 'nulle'
print(fuzzynull)

     article_id              vedette localisationde dpt_code      canton_code  \
2     PO7-00003               Abbaye   Haute-Savoie   DEP_74             Biot   
4     PO7-00006  Abbaye-de-Grandvaux           Jura   DEP_39    Saint-Laurent   
13    PO7-00015               Achard          Isère   DEP_38  Villard-de-Lans   
47    PO7-00054              Alençon          Drôme   DEP_26        Dieulefit   
92    PO7-00106             Annegray    Haute-Saône   DEP_70        Faucogney   
...         ...                  ...            ...      ...              ...   
3242  PO7-04035             Vermelle          Isère   DEP_38         Bourgoin   
3243  PO7-04037            Vernaison          Drôme   DEP_26   Bourg-de-Péage   
3264  PO7-04062               Vetraz   Haute-Savoie   DEP_74        Annemasse   
3310  PO7-04128                Ville          Isère   DEP_38   la Verpillière   
3316  PO7-04134           Villeneuve          Isère   DEP_38           Domène   

              localisationc

/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_2124/3129653776.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzzynull['method'] = 'nulle'


In [18]:
#il faut maintenant concaténer nos 3 dataframes, en respectant l'ordre original des article_id

merged_df = pd.concat([commune, fuzzy, fuzzynull], axis=0)
merged_df = merged_df.sort_values('method', ascending=True)

print(merged_df)

     article_id                 vedette localisationde dpt_code  \
0     PO7-00001                  Abbaye          Drôme   DEP_26   
2466  PO7-03100            Sainte-Croix          Loire   DEP_42   
2464  PO7-03098            Sainte-Croix        Ardèche   DEP_07   
2462  PO7-03095        Sainte-Catherine   Haute-Savoie   DEP_74   
2457  PO7-03089            Sainte-Affre      Haut-Rhin   DEP_68   
...         ...                     ...            ...      ...   
1100  PO7-01383               Fenestroz         Savoie   DEP_73   
1075  PO7-01354                   Eyzin          Isère   DEP_38   
1040  PO7-01311                   Ésery   Haute-Savoie   DEP_74   
1939  PO7-02447  Notre-Dame-de-la-Gorge   Haute-Savoie   DEP_74   
3316  PO7-04134              Villeneuve          Isère   DEP_38   

                  canton_code         localisationco insee_code method  
0                         Die                    Die      26113  exact  
2466             Rive-de-Gier                Pave

In [19]:
merged_df['origine'] = 'avecloc'

In [21]:
merged_df.to_csv("dataframe_avecloc_po7.csv", index=False)